<a href="https://colab.research.google.com/github/RC11-SkillsClass2022-23/KexinGao_Augustine/blob/main/Segment_for_highlighting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Auto-generating mask

##install

In [1]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

import os
import cv2
import json
import numpy as np
from PIL import Image
from segment_anything.utils.onnx import SamOnnxModel
from segment_anything import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator
sam = sam_model_registry["vit_h"](checkpoint="/content/drive/MyDrive/for_masks/sam_vit_h_4b8939.pth")
sam.to(device = "cuda")
mask_generator = SamAutomaticMaskGenerator(sam)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-d42g_h2u
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-d42g_h2u
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36610 sha256=49e6945978173b0ea9443245bdda0decaf3c603d34c50f73cdf4b2b4147a1c67
  Stored in directory: /tmp/pip-ephem-wheel-cache-9kt4wspj/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


##One image

In [9]:
image = '/content/drive/MyDrive/for_masks/demo/output_229.jpg'
img = cv2.imread(image)
masks = mask_generator.generate(img) #the generated mask

In [ ]:
#masks is a dictionary with several properties
#segmentation in mask is an ndarray, can be packed with .npy
#export each mask packed in a single .npy file

n = 1
for i in masks:
  name = i['segmentation']
  output_path = '/content/drive/MyDrive/for_masks/save/01/01_'+str(n)+'.npy'#stores only one array in one .npy
  np.save(output_path, name)
  n = n+1

In [ ]:
#for visualization, transform the array into png images
array = np.load('/content/drive/MyDrive/for_masks/save/01/01_1.npy')
image = Image.fromarray(array)
image.save('/content/drive/MyDrive/for_masks/save/01/01_1.png')

In [ ]:
#extract the masked object in the image

# Load the image and mask
img = cv2.imread("image.jpg")
mask = cv2.imread("mask.jpg", 0) # 0 means load as grayscale

# Apply the mask to the image
masked_img = cv2.bitwise_and(img, img, mask=mask)

# Export the masked image
cv2.imwrite("masked_image.jpg", masked_img)


In [3]:
#Extract the masked object in the image
img = cv2.imread('/content/drive/MyDrive/for_masks/demo/output_229.jpg')
mask = cv2.imread('/content/drive/MyDrive/for_masks/save_image/1/1_3.png', 0) # 0 means load as grayscale

# Apply the mask to the image
masked_img = cv2.bitwise_and(img, img, mask=mask)

# Export the masked image
cv2.imwrite('/content/drive/MyDrive/for_masks/demo/output_229.png', masked_img)


True

###is there a need for packing masks in one file?

In [ ]:
#is there a need for packing masks in one file?
#not needed
np.savez('/content/drive/MyDrive/for_masks/data.npz',)#names of all the arrays need packing

##Batch processing

In [9]:
def Segmentation(folder):
  #load the images
  a = 1
  image_list = os.listdir(folder)
  images = []
  for image in image_list:
    images.append(os.path.join(folder, image))
  for image in images:
    img = cv2.imread(image)
    masks = mask_generator.generate(img)
    #download the masks
    n = 1
    for i in masks:
      name = i['segmentation']
      if n == 1:
        output_folder = '/content/drive/MyDrive/for_masks/save_npy/'+str(a)
        output_folder_image = '/content/drive/MyDrive/for_masks/save_image/'+str(a)
        output_folder_masked = '/content/drive/MyDrive/for_masks/save_masked/'+str(a)  
        os.makedirs(output_folder)
        os.makedirs(output_folder_image)
        os.makedirs(output_folder_masked)
      #save .npy file
      output_path = output_folder+'/'+str(a)+'_'+str(n)+'.npy'
      np.save(output_path, name)
      #save .png file
      array = np.load(output_path)
      mask_image = Image.fromarray(array)
      output_path_image = output_folder_image+'/'+str(a)+'_'+str(n)+'.png'
      mask_image.save(output_path_image)
      #save the masked image as png
      mask = cv2.imread(output_path_image, 0)
      masked_img = cv2.bitwise_and(img, img, mask=mask)
      cv2.imwrite(output_folder_masked+'/'+str(a)+'_'+str(n)+'.png', masked_img)
      n = n+1
    a = a+1

In [10]:
#input a folder of frames
folder = '/content/drive/MyDrive/for_masks/demo'
Segmentation(folder)

AttributeError: ignored